In [2]:
import __init__

# pek212202_0000000005
sql = """select i.zzitemid, year(from_unixtime(i.startep)) as period, (i.y6-i.y5) as result from anal_zzitems_D_5_5 i
inner join anal_zzkmgroups_D_5_5 g on i.zzitemid = g.zzitemid
where g.km_id = 'pek212202_0000000005'; 
"""

import pandas as pd
import numpy as np
import scipy
from db.mydf import MyDf
mydf = MyDf()
df = mydf.read_sql(sql)
df
#df['date'] = pd.to_datetime(df['startep'],unit='s')
#df['year'] = df['date'].dt.strftime('%Y')

,zzitemid,period,result
0,300,2014,0.722240
1,893,2017,-0.143115
2,1042,2012,-0.119483
3,1148,2016,0.133314
4,1168,2017,-0.048802
...,...,...,...
767,468499,2017,-0.333333
768,468553,2018,-0.571429
769,468735,2014,-0.142857
770,469410,2019,-1.874872


In [3]:

udf = df[df["result"]>0.1][["period", "result"]].rename(columns={"result": "up"})
muresult = scipy.stats.trim_mean(udf["up"],0.4)
print(muresult)

ddf = df[df["result"]<-0.1][["period", "result"]].rename(columns={"result": "down"})
mdresult = scipy.stats.trim_mean(ddf["down"],0.4)
print(mdresult)

dfp = df[["period", "result"]].groupby("period").count().rename(columns={"result": "total"})
udfp = udf.groupby("period").count()
ddfp = ddf.groupby("period").count()

adfp = pd.merge(dfp, udfp, on=["period"], how="outer").replace(np.nan, 0)
adfp = pd.merge(adfp, ddfp, on=["period"], how="outer").replace(np.nan, 0)
total = len(adfp)
ucnt = adfp[adfp["up"]/adfp["total"] > 0.5]["total"].count()
dcnt = adfp[adfp["down"]/adfp["total"] > 0.5]["total"].count()

score1 = max(ucnt, dcnt)/total
print(score1)
score2 = max(udf["up"].count(),ddf["down"].count())/(df["result"].count())
print(score2)

#adfp

0.4016040389483072
-0.34552494489898283
0.2
0.4805699481865285
